In this tutorial, we will prepare Milvus database for storing and searching nodes and edges of a graph.

In particular, we are using PrimeKG multimodal data from the BioBridge project.

In [1]:
# Load necessary libraries
import os
import glob
import hydra
import cudf
import cupy as cp
from pymilvus import (
    db,
    connections,
    FieldSchema,
    CollectionSchema,
    DataType,
    Collection,
    utility,
    MilvusClient
)
from tqdm import tqdm
import time

### Loading BioBridge-PrimeKG Multimodal Data

First, we need to get the path to the directory containing the parquet files of nodes and edges.

For nodes and edges, we have a separate folder that contains its enrichment and embeddings.

In [2]:
# Load hydra configuration
with hydra.initialize(version_base=None, config_path="../../../aiagents4pharma/talk2knowledgegraphs/configs"):
    cfg = hydra.compose(
        config_name="config", overrides=["tools/multimodal_subgraph_extraction=default"]
    )
    cfg = cfg.tools.multimodal_subgraph_extraction
cfg

{'_target_': 'talk2knowledgegraphs.tools.multimodal_subgraph_extraction', 'ollama_embeddings': ['nomic-embed-text'], 'temperature': 0.1, 'streaming': False, 'topk': 5, 'topk_e': 5, 'cost_e': 0.5, 'c_const': 0.01, 'root': -1, 'num_clusters': 1, 'pruning': 'gw', 'verbosity_level': 0, 'node_id_column': 'node_id', 'node_attr_column': 'node_attr', 'edge_src_column': 'edge_src', 'edge_attr_column': 'edge_attr', 'edge_dst_column': 'edge_dst', 'node_colors_dict': {'gene/protein': '#6a79f7', 'molecular_function': '#82cafc', 'cellular_component': '#3f9b0b', 'biological_process': '#c5c9c7', 'drug': '#c4a661', 'disease': '#80013f'}, 'biobridge': {'source': '/mnt/blockstorage/biobridge_multimodal/', 'node_type': ['gene/protein', 'molecular_function', 'cellular_component', 'biological_process', 'drug', 'disease']}}

In [3]:
# You can set the source directory for biobridge data here
cfg.biobridge.source = "/mnt/blockstorage/biobridge_multimodal"

In [4]:
%%time

# Loop over nodes and edges
graph_dict = {}
for element in ["nodes", "edges"]:
    # Make an empty dictionary for each folder
    graph_dict[element] = {}
    for stage in ["enrichment", "embedding"]:
        print(element, stage)
        # Create the file pattern for the current subfolder
        file_list = glob.glob(os.path.join(cfg.biobridge.source,
                                           element,
                                           stage, '*.parquet.gzip'))
        print(file_list)
        # Read and concatenate all dataframes in the folder
        # Except the edges embedding, which is too large to read in one go
        # We are using a chunk size to read the edges embedding in smaller parts instead
        if element == "edges" and stage == "embedding":
            # For edges embedding, only read two columns: triplet_index and edge_emb
            # graph_dict[element][stage] = cudf.concat([cudf.read_parquet(f, columns=["triplet_index", "edge_emb"]) for f in file_list[:2]], ignore_index=True)
            # Loop by chunks
            # file_list = file_list[:2]
            chunk_size = 5
            graph_dict[element][stage] = []
            for i in range(0, len(file_list), chunk_size):
                chunk_files = file_list[i:i+chunk_size]
                chunk_df = cudf.concat([cudf.read_parquet(f, columns=["triplet_index", "edge_emb"]) for f in chunk_files], ignore_index=True)
                graph_dict[element][stage].append(chunk_df)
        else:
            # For nodes and edges enrichment, read and concatenate all dataframes in the folder
            # This includes the nodes embedding, which is small enough to read in one go
            graph_dict[element][stage] = cudf.concat([cudf.read_parquet(f) for f in file_list], ignore_index=True)

nodes enrichment
['/mnt/blockstorage/biobridge_multimodal/nodes/enrichment/drug.parquet.gzip', '/mnt/blockstorage/biobridge_multimodal/nodes/enrichment/cellular_component.parquet.gzip', '/mnt/blockstorage/biobridge_multimodal/nodes/enrichment/gene_protein.parquet.gzip', '/mnt/blockstorage/biobridge_multimodal/nodes/enrichment/disease.parquet.gzip', '/mnt/blockstorage/biobridge_multimodal/nodes/enrichment/biological_process.parquet.gzip', '/mnt/blockstorage/biobridge_multimodal/nodes/enrichment/molecular_function.parquet.gzip']
nodes embedding
['/mnt/blockstorage/biobridge_multimodal/nodes/embedding/drug.parquet.gzip', '/mnt/blockstorage/biobridge_multimodal/nodes/embedding/cellular_component.parquet.gzip', '/mnt/blockstorage/biobridge_multimodal/nodes/embedding/gene_protein.parquet.gzip', '/mnt/blockstorage/biobridge_multimodal/nodes/embedding/disease.parquet.gzip', '/mnt/blockstorage/biobridge_multimodal/nodes/embedding/biological_process.parquet.gzip', '/mnt/blockstorage/biobridge_mu

In [5]:
# Get nodes enrichment and embedding dataframes
nodes_enrichment_df = graph_dict['nodes']['enrichment']
nodes_embedding_df = graph_dict['nodes']['embedding']

# Get edges enrichment and embedding dataframes
edges_enrichment_df = graph_dict['edges']['enrichment']
edges_embedding_df = graph_dict['edges']['embedding'] # !!consisted of a list of dataframes!!

In [6]:
# Merge nodes enrichment and embedding dataframes
merged_nodes_df = nodes_enrichment_df.merge(
    nodes_embedding_df[["node_id", "desc_emb", "feat_emb"]],
    on="node_id",
    how="left"
)
# del nodes_enrichment_df, nodes_embedding_df  # Free memory

### Setup Milvus Database

In [4]:
# Configuration for Milvus
milvus_host = "localhost"
milvus_port = "19530"
milvus_uri = "http://localhost:19530"
milvus_token = "root:Milvus"
milvus_user = "root"
milvus_password = "Milvus"
milvus_database = "t2kg_primekg"

In [5]:
# Connect to Milvus
connections.connect(
    alias="default",
    host=milvus_host,
    port=milvus_port,
    user=milvus_user,
    password=milvus_password
)

In [6]:
# Check if the database exists, create if it doesn't
if milvus_database not in db.list_database():
    db.create_database(milvus_database)

# Switch to the desired database
db.using_database(milvus_database)

In [10]:
# List all collections
for coll in utility.list_collections():
    print(f"Collection: {coll}")

    # Load the collection to get stats
    collection = Collection(name=coll)
    print(collection.num_entities)

In [11]:
# A helper function to chunk the data into smaller parts
# Utility: chunk generator
def chunked(data_list, chunk_size):
    for i in range(0, len(data_list), chunk_size):
        yield data_list[i:i + chunk_size]

#### Building Node Collection (Description Embedding)

In [12]:
%%time

# Configuration for Milvus collection
node_coll_name = f"{milvus_database}_nodes"

# Define schema for the collection
# Leave out the feat and feat_emb fields for now
desc_emb_dim = len(merged_nodes_df.iloc[0]['desc_emb'].to_arrow().to_pylist()[0])
node_fields = [
    FieldSchema(name="node_index", dtype=DataType.INT64, is_primary=True),
    FieldSchema(name="node_id", dtype=DataType.VARCHAR, max_length=1024),
    FieldSchema(name="node_name", dtype=DataType.VARCHAR, max_length=1024),
    FieldSchema(name="node_type", dtype=DataType.VARCHAR, max_length=1024),
    FieldSchema(name="desc", dtype=DataType.VARCHAR, max_length=40960),
    FieldSchema(name="desc_emb", dtype=DataType.FLOAT_VECTOR, dim=desc_emb_dim),
]
schema  = CollectionSchema(fields=node_fields, description=f"Schema for collection {node_coll_name}")

# Create collection if it doesn't exist
if not utility.has_collection(node_coll_name):
    collection = Collection(name=node_coll_name, schema=schema)
else:
    collection = Collection(name=node_coll_name)

# Create indexes
collection.create_index(
    field_name="node_index",
    index_params={"index_type": "STL_SORT"}, # STL_SORT
    index_name="node_index_index"
)

collection.create_index(
    field_name="desc_emb",
    index_params={"index_type": "GPU_CAGRA", "metric_type": "COSINE"}, # AUTOINDEX
    index_name="desc_emb_index"
)

# Prepare data for insertion
data = [
    merged_nodes_df["node_index"].to_arrow().to_pylist(),
    merged_nodes_df["node_id"].to_arrow().to_pylist(),
    merged_nodes_df["node_name"].to_arrow().to_pylist(),
    merged_nodes_df["node_type"].to_arrow().to_pylist(),
    merged_nodes_df["desc"].to_arrow().to_pylist(),
    cp.asarray(merged_nodes_df["desc_emb"].list.leaves).astype(cp.float32)
        .reshape(merged_nodes_df.shape[0], -1)
        .tolist(),
]

# Insert data in batches
batch_size = 500
total = len(data[0])
for i in tqdm(range(0, total, batch_size)):
    batch = [
        col[i:i+batch_size] for col in data
    ]
    collection.insert(batch)

# Flush to persist data
collection.flush()

# Get collection stats
print(collection.num_entities)

100%|██████████| 170/170 [01:52<00:00,  1.51it/s]


84981
CPU times: user 21.2 s, sys: 2.03 s, total: 23.3 s
Wall time: 1min 59s


In [13]:
# List all collections
for coll in utility.list_collections():
    print(f"Collection: {coll}")

    # Load the collection to get stats
    collection = Collection(name=coll)
    print(collection.num_entities)

Collection: t2kg_primekg_nodes
84981


In [14]:
%%time

# Assume node_coll_name is defined and collection exists
collection = Collection(node_coll_name)

# Load the collection into memory before query
collection.load()

# Query by expr on node_index
expr = "node_index == 13814"
output_fields = ["node_index", "node_id", "node_name", "node_type", "desc", "desc_emb"]

results = collection.query(expr, output_fields=output_fields)

print(results)

data: ["{'node_type': 'drug', 'desc': 'Copper belongs to drug node. Copper is a transition metal and a trace element in the body. It is important to the function of many enzymes including cytochrome c oxidase, monoamine oxidase and superoxide dismutase. Copper is commonly used in contraceptive intrauterine devices (IUD). Copper is absorbed from the gut via high affinity copper uptake protein and likely through low affinity copper uptake protein and natural resistance-associated macrophage protein-2. It is believed that copper is reduced to the Cu1+ form prior to transport. Once inside the enterocyte, it is bound to copper transport protein ATOX1 which shuttles the ion to copper transporting ATPase-1 on the golgi membrane which take up copper into the golgi apparatus. Once copper has been secreted by enterocytes into the systemic circulation it remain largely bound by ceruloplasmin (65-90%), albumin (18%), and alpha 2-macroglobulin (12%).  Copper is nearly entirely bound by ceruloplasmi

In [15]:
%%time

# Assume node_coll_name is defined and collection exists
collection = Collection(node_coll_name)

# Load the collection into memory before query
collection.load()

# Query by expr on node_index
expr = "node_index in [13814, 13815]"
output_fields = ["node_index", "node_id", "node_name", "node_type", "desc", "desc_emb"]

results = collection.query(expr, output_fields=output_fields)

print(results)

data: ["{'node_type': 'drug', 'desc': 'Copper belongs to drug node. Copper is a transition metal and a trace element in the body. It is important to the function of many enzymes including cytochrome c oxidase, monoamine oxidase and superoxide dismutase. Copper is commonly used in contraceptive intrauterine devices (IUD). Copper is absorbed from the gut via high affinity copper uptake protein and likely through low affinity copper uptake protein and natural resistance-associated macrophage protein-2. It is believed that copper is reduced to the Cu1+ form prior to transport. Once inside the enterocyte, it is bound to copper transport protein ATOX1 which shuttles the ion to copper transporting ATPase-1 on the golgi membrane which take up copper into the golgi apparatus. Once copper has been secreted by enterocytes into the systemic circulation it remain largely bound by ceruloplasmin (65-90%), albumin (18%), and alpha 2-macroglobulin (12%).  Copper is nearly entirely bound by ceruloplasmi

In [16]:
%%time

vector_to_search = merged_nodes_df["desc_emb"].loc[2]


# Assume node_coll_name is defined and collection exists
collection = Collection(node_coll_name)

# Load the collection into memory before query
collection.load()

# Vector similarity search in Milvus
vector_to_search = merged_nodes_df["desc_emb"].iloc[2]  # or .loc[2], both work
search_params = {"metric_type": "COSINE"}
results = collection.search(
    data=[vector_to_search],
    anns_field="desc_emb",
    param=search_params,
    limit=10,
    output_fields=["node_id", "node_name"]
)

results

CPU times: user 15.3 ms, sys: 1.18 ms, total: 16.5 ms
Wall time: 285 ms


data: [[{'node_index': 11920, 'distance': 1.0000001192092896, 'entity': {'node_id': 'FHAD1_(12058)', 'node_name': 'FHAD1'}}, {'node_index': 28257, 'distance': 0.9123432636260986, 'entity': {'node_id': 'PIH1D3_(35196)', 'node_name': 'PIH1D3'}}, {'node_index': 10809, 'distance': 0.90804123878479, 'entity': {'node_id': 'FHOD1_(10917)', 'node_name': 'FHOD1'}}, {'node_index': 11129, 'distance': 0.9008122682571411, 'entity': {'node_id': 'FHIP2A_(11243)', 'node_name': 'FHIP2A'}}, {'node_index': 52440, 'distance': 0.8989576101303101, 'entity': {'node_id': 'FHIP2B_(83697)', 'node_name': 'FHIP2B'}}, {'node_index': 66576, 'distance': 0.8985256552696228, 'entity': {'node_id': "protein-FAD linkage via 1'-(8alpha-FAD)-L-histidine_(108484)", 'node_name': "protein-FAD linkage via 1'-(8alpha-FAD)-L-histidine"}}, {'node_index': 11610, 'distance': 0.8984012603759766, 'entity': {'node_id': 'FHOD3_(11738)', 'node_name': 'FHOD3'}}, {'node_index': 7602, 'distance': 0.8982304334640503, 'entity': {'node_id': '

In [17]:
merged_nodes_df.loc[2]

,node_index,primekg_node_index,node_id,node_name,node_type,desc,feat,desc_emb,feat_emb
2,11920,12058,FHAD1_(12058),FHAD1,gene/protein,FHAD1 belongs to gene/protein node. FHAD1 is f...,MKAYLKSAEGFFVLNKSTTIGRHENSDLVLQSPDIDNHHALIEYNE...,"[-0.004289147, -0.0060087284, -0.02683332, -0....","[-0.04512454941868782, -0.14413529634475708, 0..."


#### Building Node Collection (Node Type-specific Embedding)

Note that nodes information of the PrimeKG data is different for each node type, 
we are going to build a separate collection for each node type.

We will use the node type as the collection name.

In [18]:
%%time

# Loop over group enrichment nodes by node_type
for node_type, nodes_df in tqdm(merged_nodes_df.groupby('node_type')):
    print(f"Processing node type: {node_type}")

    # Milvus collection name for this node_type
    node_coll_name = f"{milvus_database}_nodes_{node_type.replace('/', '_')}"

    # Define collection schema
    feat_emb_dim = len(nodes_df.iloc[0]['feat_emb'].to_arrow().to_pylist()[0])
    node_fields = [
        FieldSchema(name="node_index", dtype=DataType.INT64, is_primary=True, auto_id=False),
        FieldSchema(name="node_id", dtype=DataType.VARCHAR, max_length=1024),
        FieldSchema(name="node_name", dtype=DataType.VARCHAR, max_length=1024),
        FieldSchema(name="node_type", dtype=DataType.VARCHAR, max_length=1024),
        FieldSchema(name="feat", dtype=DataType.VARCHAR, max_length=40960),
        FieldSchema(name="feat_emb", dtype=DataType.FLOAT_VECTOR, dim=feat_emb_dim),
    ]
    schema = CollectionSchema(fields=node_fields, description=f"schema for collection {node_coll_name}")

    # Create collection if not exists
    if not utility.has_collection(node_coll_name):
        collection = Collection(name=node_coll_name, schema=schema)
    else:
        collection = Collection(name=node_coll_name)

    # Create index for node_index field (scalar)
    collection.create_index(
        field_name="node_index",
        index_params={"index_type": "STL_SORT"},
        index_name="node_index_index"
    )

    # Create index for feat_emb (vector)
    collection.create_index(
        field_name="feat_emb",
        index_params={"index_type": "GPU_CAGRA", "metric_type": "COSINE"}, # AUTOINDEX
        index_name="feat_emb_index"
    )

    # Prepare data for insertion
    # Columns must be lists of values in order matching schema fields
    data = [
        nodes_df["node_index"].to_arrow().to_pylist(),
        nodes_df["node_id"].to_arrow().to_pylist(),
        nodes_df["node_name"].to_arrow().to_pylist(),
        nodes_df["node_type"].to_arrow().to_pylist(),
        nodes_df["feat"].to_arrow().to_pylist(),
        cp.asarray(nodes_df["feat_emb"].list.leaves).astype(cp.float32)
            .reshape(nodes_df.shape[0], -1)
            .tolist(),
    ]

    # Batch insert data in chunks
    batch_size = 500
    total_rows = len(data[0])
    for i in tqdm(range(0, total_rows, batch_size)):
        batch = [col[i:i + batch_size] for col in data]
        collection.insert(batch)

    # Flush the collection to ensure data is persisted
    collection.flush()

    # Print collection stats (number of entities and segment info)
    stats = collection.num_entities
    print(f"Collection {node_coll_name} stats:")
    print(stats)

  0%|          | 0/6 [00:00<?, ?it/s]

Processing node type: biological_process


 17%|█▋        | 1/6 [00:41<03:28, 41.61s/it]

Collection t2kg_primekg_nodes_biological_process stats:
27409
Processing node type: cellular_component


 33%|███▎      | 2/6 [00:49<01:27, 21.99s/it]

...

Collection t2kg_primekg_nodes_drug stats:
6759
Processing node type: gene/protein


 83%|████████▎ | 5/6 [01:53<00:22, 22.03s/it]

Collection t2kg_primekg_nodes_gene_protein stats:
18797
Processing node type: molecular_function


100%|██████████| 6/6 [02:10<00:00, 21.78s/it]

Collection t2kg_primekg_nodes_molecular_function stats:
10951
CPU times: user 25.5 s, sys: 2.86 s, total: 28.3 s
Wall time: 2min 10s


In [19]:
# List all collections
for coll in utility.list_collections():
    print(f"Collection: {coll}")

    # Load the collection to get stats
    collection = Collection(name=coll)
    print(collection.num_entities)

Collection: t2kg_primekg_nodes_drug
6759
Collection: t2kg_primekg_nodes_gene_protein
18797
Collection: t2kg_primekg_nodes_molecular_function
10951
Collection: t2kg_primekg_nodes
84981
Collection: t2kg_primekg_nodes_biological_process
27409
Collection: t2kg_primekg_nodes_cellular_component
4011
Collection: t2kg_primekg_nodes_disease
17054


In [20]:
%%time

# Assume node_coll_name is defined and collection exists
collection = Collection('t2kg_primekg_nodes_gene_protein')

# Load the collection into memory before query
collection.load()

# Vector similarity search in Milvus
vector_to_search = merged_nodes_df["feat_emb"].iloc[2]  # or .loc[2], both work
search_params = {"metric_type": "COSINE"}
results = collection.search(
    data=[vector_to_search],
    anns_field="feat_emb",
    param=search_params,
    limit=10,
    output_fields=["node_id", "node_name"]
)

results

CPU times: user 18.8 ms, sys: 2.39 ms, total: 21.2 ms
Wall time: 2.68 s


data: [[{'node_index': 11920, 'distance': 0.9999998807907104, 'entity': {'node_id': 'FHAD1_(12058)', 'node_name': 'FHAD1'}}, {'node_index': 2175, 'distance': 0.9948822259902954, 'entity': {'node_id': 'CEP63_(2181)', 'node_name': 'CEP63'}}, {'node_index': 11540, 'distance': 0.9939631819725037, 'entity': {'node_id': 'CNTLN_(11667)', 'node_name': 'CNTLN'}}, {'node_index': 52366, 'distance': 0.9933797717094421, 'entity': {'node_id': 'CCDC30_(77529)', 'node_name': 'CCDC30'}}, {'node_index': 10812, 'distance': 0.993129312992096, 'entity': {'node_id': 'CEP85_(10920)', 'node_name': 'CEP85'}}, {'node_index': 511, 'distance': 0.9928526282310486, 'entity': {'node_id': 'PDE4DIP_(511)', 'node_name': 'PDE4DIP'}}, {'node_index': 10647, 'distance': 0.9928280115127563, 'entity': {'node_id': 'DEUP1_(10750)', 'node_name': 'DEUP1'}}, {'node_index': 51746, 'distance': 0.992749035358429, 'entity': {'node_id': 'LUZP2_(60155)', 'node_name': 'LUZP2'}}, {'node_index': 114, 'distance': 0.9925759434700012, 'entit

In [21]:
# Check the ground truth for the search
merged_nodes_df.loc[2]

,node_index,primekg_node_index,node_id,node_name,node_type,desc,feat,desc_emb,feat_emb
2,11920,12058,FHAD1_(12058),FHAD1,gene/protein,FHAD1 belongs to gene/protein node. FHAD1 is f...,MKAYLKSAEGFFVLNKSTTIGRHENSDLVLQSPDIDNHHALIEYNE...,"[-0.004289147, -0.0060087284, -0.02683332, -0....","[-0.04512454941868782, -0.14413529634475708, 0..."


In [22]:
# Get node indices from the results
[n['node_index'] for n in results[0]]

[11920, 2175, 11540, 52366, 10812, 511, 10647, 51746, 114, 2939]

In [23]:
# Get the cosine similarity scores
[n['distance'] for n in results[0]]

[0.9999998807907104,
 0.9948822259902954,
 0.9939631819725037,
 0.9933797717094421,
 0.993129312992096,
 0.9928526282310486,
 0.9928280115127563,
 0.992749035358429,
 0.9925759434700012,
 0.9923074245452881]

#### Building Edge Collection

Subsquently, we are also building the edges collection in Milvus.

Note that the edges information of PrimeKG has massive records, so once again we are chunking the data to avoid memory issues.

In [24]:
%%time

# Define collection name
edge_coll_name = f"{milvus_database}_edges"

# Define schema
edge_fields = [
    FieldSchema(name="triplet_index", dtype=DataType.INT64, is_primary=True, auto_id=False),
    FieldSchema(name="head_id", dtype=DataType.VARCHAR, max_length=1024),
    FieldSchema(name="head_index", dtype=DataType.INT64),
    FieldSchema(name="tail_id", dtype=DataType.VARCHAR, max_length=1024),
    FieldSchema(name="tail_index", dtype=DataType.INT64),
    FieldSchema(name="edge_type", dtype=DataType.VARCHAR, max_length=1024),
    FieldSchema(name="display_relation", dtype=DataType.VARCHAR, max_length=1024),
    FieldSchema(name="feat", dtype=DataType.VARCHAR, max_length=40960),
    FieldSchema(name="feat_emb", dtype=DataType.FLOAT_VECTOR, dim=1536),
]
edge_schema = CollectionSchema(fields=edge_fields, description="Schema for edges collection")

# Create collection if not exists
if not utility.has_collection(edge_coll_name):
    collection = Collection(name=edge_coll_name, schema=edge_schema)
else:
    collection = Collection(name=edge_coll_name)

# Create indexes
collection.create_index(field_name="triplet_index", index_params={"index_type": "STL_SORT"}, index_name="triplet_index_index")
collection.create_index(field_name="head_index", index_params={"index_type": "STL_SORT"}, index_name="head_index_index")
collection.create_index(field_name="tail_index", index_params={"index_type": "STL_SORT"}, index_name="tail_index_index")
collection.create_index(field_name="feat_emb", index_params={"index_type": "GPU_CAGRA", "metric_type": "COSINE"}, index_name="feat_emb_index") # AUTOINDEX

# Iterate over chunked edges embedding df
for edges_df in tqdm(edges_embedding_df):
    # Merge enrichment with embedding
    merged_edges_df = edges_enrichment_df.merge(
        edges_df[["triplet_index", "edge_emb"]],
        on="triplet_index",
        how="inner"
    )

    # Prepare data fields in column-wise format
    data = [
        merged_edges_df["triplet_index"].to_arrow().to_pylist(),
        merged_edges_df["head_id"].to_arrow().to_pylist(),
        merged_edges_df["head_index"].to_arrow().to_pylist(),
        merged_edges_df["tail_id"].to_arrow().to_pylist(),
        merged_edges_df["tail_index"].to_arrow().to_pylist(),
        merged_edges_df["edge_type_str"].to_arrow().to_pylist(),
        merged_edges_df["display_relation"].to_arrow().to_pylist(),
        merged_edges_df["feat"].to_arrow().to_pylist(),
        cp.asarray(merged_edges_df["edge_emb"].list.leaves).astype(cp.float32)
            .reshape(merged_edges_df.shape[0], -1)
            .tolist(),
    ]

    # Insert in chunks
    batch_size = 500
    for i in tqdm(range(0, len(data[0]), batch_size)):
        batch_data = [d[i:i+batch_size] for d in data]
        collection.insert(batch_data)

    # Flush to ensure persistence
    collection.flush()

    # Print collection stats
    print(collection.num_entities)
    
    time.sleep(5)  # Sleep to avoid overwhelming the server


100%|██████████| 500/500 [05:35<00:00,  1.49it/s]


250000


100%|██████████| 500/500 [05:38<00:00,  1.48it/s]


500000


100%|██████████| 500/500 [05:36<00:00,  1.48it/s]


...

3704610


100%|██████████| 400/400 [04:38<00:00,  1.44it/s]]


3904610


100%|██████████| 16/16 [1:38:37<00:00, 369.84s/it]

CPU times: user 17min 16s, sys: 1min 50s, total: 19min 7s
Wall time: 1h 38min 39s


In [7]:
# List all collections
for coll in utility.list_collections():
    print(f"Collection: {coll}")

    # Load the collection to get stats
    collection = Collection(name=coll)
    print(collection.num_entities)

Collection: t2kg_primekg_nodes
84981
Collection: t2kg_primekg_nodes_biological_process
27409
Collection: t2kg_primekg_nodes_cellular_component
4011
Collection: t2kg_primekg_nodes_disease
17054
Collection: t2kg_primekg_nodes_drug
6759
Collection: t2kg_primekg_nodes_gene_protein
18797
Collection: t2kg_primekg_nodes_molecular_function
10951
Collection: t2kg_primekg_edges
3904610


In [29]:
%%time

# Assume node_coll_name is defined and collection exists
collection = Collection('t2kg_primekg_edges')

# Load the collection into memory before query
collection.load()

# Query by expr on triplet_index
expr = "triplet_index == 0"
output_fields = ["triplet_index", "head_id", "tail_id", "edge_type", "feat", "feat_emb"]

results = collection.query(expr, output_fields=output_fields)
results

CPU times: user 3.66 ms, sys: 98 μs, total: 3.75 ms
Wall time: 13.8 ms


data: ["{'tail_id': 'KIF15_(8889)', 'edge_type': 'gene/protein|ppi|gene/protein', 'feat': 'PHYHIP (gene/protein) has a direct relationship of protein_protein:ppi with KIF15 (gene/protein).', 'feat_emb': [np.float32(-0.01934238), np.float32(0.0011752498), np.float32(0.004431808), np.float32(-0.033904973), np.float32(-0.01556057), np.float32(0.011148459), np.float32(0.0011645806), np.float32(0.028704984), np.float32(0.012691384), np.float32(-0.005199988), np.float32(0.020970657), np.float32(0.01423431), np.float32(0.009421695), np.float32(0.0033714569), np.float32(-0.0027230997), np.float32(-0.0106954295), np.float32(0.015586833), np.float32(-0.015100975), np.float32(0.01833127), np.float32(0.014812087), np.float32(-0.008581293), np.float32(0.019014098), np.float32(0.005584078), np.float32(-0.036977693), np.float32(0.010511592), np.float32(0.034535274), np.float32(0.010833308), np.float32(-0.026236303), np.float32(-0.025107013), np.float32(-0.037502944), np.float32(0.027575694), np.float

In [30]:
# Check the ground truth for the search
results[0]['triplet_index'], results[0]['head_id'], results[0]['tail_id'], results[0]['edge_type'], results[0]['feat']

(0,
 'PHYHIP_(0)',
 'KIF15_(8889)',
 'gene/protein|ppi|gene/protein',
 'PHYHIP (gene/protein) has a direct relationship of protein_protein:ppi with KIF15 (gene/protein).')

In [31]:
%%time

# Assume node_coll_name is defined and collection exists
collection = Collection('t2kg_primekg_edges')

# Load the collection into memory before query
collection.load()

# Vector similarity search in Milvus
vector_to_search = np.array(results[0]['feat_emb']).tolist() # merged_edges_df["edge_emb"].iloc[0]
search_params = {"metric_type": "COSINE"}
results = collection.search(
    data=[vector_to_search],
    anns_field="feat_emb",
    param=search_params,
    limit=10,
    output_fields=["head_id", "tail_id", "edge_type", "feat"]
)
results

CPU times: user 19.8 ms, sys: 7.31 ms, total: 27.1 ms
Wall time: 37.4 s


data: [[{'triplet_index': 0, 'distance': 1.000000238418579, 'entity': {'edge_type': 'gene/protein|ppi|gene/protein', 'feat': 'PHYHIP (gene/protein) has a direct relationship of protein_protein:ppi with KIF15 (gene/protein).', 'head_id': 'PHYHIP_(0)', 'tail_id': 'KIF15_(8889)'}}, {'triplet_index': 3069556, 'distance': 0.9815623760223389, 'entity': {'edge_type': 'gene/protein|ppi|gene/protein', 'feat': 'KIF15 (gene/protein) has a direct relationship of protein_protein:ppi with PHYHIP (gene/protein).', 'head_id': 'KIF15_(8889)', 'tail_id': 'PHYHIP_(0)'}}, {'triplet_index': 93582, 'distance': 0.9684444069862366, 'entity': {'edge_type': 'gene/protein|ppi|gene/protein', 'feat': 'PHYHIP (gene/protein) has a direct relationship of protein_protein:ppi with PRKD2 (gene/protein).', 'head_id': 'PHYHIP_(0)', 'tail_id': 'PRKD2_(9221)'}}, {'triplet_index': 305788, 'distance': 0.9657970070838928, 'entity': {'edge_type': 'gene/protein|ppi|gene/protein', 'feat': 'PHYHIP (gene/protein) has a direct relat

In [32]:
# Get node indices from the results
[n['triplet_index'] for n in results[0]]

[0, 3069556, 93582, 305788, 300682, 3154085, 3201926, 3334423, 110476, 307994]

In [33]:
# Get the cosine similarity scores
[n['distance'] for n in results[0]]

[1.000000238418579,
 0.9815623760223389,
 0.9684444069862366,
 0.9657970070838928,
 0.9599261283874512,
 0.9597105979919434,
 0.9593513607978821,
 0.9588443636894226,
 0.9586011171340942,
 0.9583001136779785]